In [142]:
import numpy as np
import plotly.express as px
from tqdm import tqdm

# Dataset Generator

Vytvoríme si dataset, ktorý bude mať 2 príznaky X, Y súradnice a 1 výstupnú triedu, ktorá bude určovať, či sa bod nachádza v kruhu alebo nie.
Tvar dát musí byť (Batch, Features).

`draw_circles` - Pomocná funkcia na vykreslenie dát.

In [143]:
def dataset_Circles(m=10, radius=0.7, noise=0.0, verbose=False):

    # Hodnoty X budu v intervale <-1; 1>
    X = (np.random.rand(m, 2) * 2.0) - 1.0
    if (verbose): print('X: \n', X, '\n')

    # Element-wise nasobenie nahodnym sumom
    N = (np.random.rand(m, 2)-0.5) * noise
    if (verbose): print('N: \n', N, '\n')
    Xnoise = X + N
    if (verbose): print('Xnoise: \n', Xnoise, '\n')

    # Spocitame polomer
    # Element-wise druha mocnina
    XSquare = Xnoise ** 2
    if (verbose): print('XSquare: \n', XSquare, '\n')

    # Spocitame podla druhej osi. Ziskame (m, 1) array.
    RSquare = np.sum(XSquare, axis=1, keepdims=True)
    if (verbose): print('RSquare: \n', RSquare, '\n')
    R = np.sqrt(RSquare)
    if (verbose): print('R: \n', R, '\n')

    # Y bude 1, ak je polomer vacsi ako argument radius
    Y = (R > radius).astype(float)
    if (verbose): print('Y: \n', Y, '\n')

    # Vratime X, Y
    return X, Y

def draw_circles(x, y):
    if x.shape[0] == 2:
        fig = px.scatter(x=x[0], y=x[1], color=np.array(y[0], dtype=str), width=700, height=700)
    elif x.shape[1] == 2:
        xx = x.reshape(-1,2).T
        yy = y.reshape(1,-1)
        fig = px.scatter(x=xx[0], y=xx[1], color=np.array(yy[0], dtype=str), width=700, height=700)
    else:
        return
    fig.show()

x, y = dataset_Circles(1024, noise=0.2)
print(x.shape)
print(y.shape)
draw_circles(x, y)

(1024, 2)
(1024, 1)


# Import najdôležitejších knižníc

In [144]:
import torch
import torch.nn as nn
import torchmetrics.classification as metrics

# Definícia našej siete

Najprv potrebujeme definovať architektúru našej siete.

Vytvoríme jednoduchú sieť, ktorá bude mať 2 skryté vrstvy a 1 výstupnú vrstvu s 1 neurónom, keďže robíme binárnu klasifikáciu.

Medzi vrstvami dáme aktivačnú funkciu ReLU, ktorá je najčastejšie používaná pri neurónových sieťach.

Na výstupe použijeme aktivačnú funkciu sigmoid, ktorá nám vráti hodnotu medzi 0 a 1.

Môžeme vyskúšať rôzne aktivačné funkcie v rôznych vrstvách a uvidíme ich vplyv na tréning.

Každej sieti potrebujeme definovať funkciu `forward`, ktorá aplikuje dopredný chod a vypočíta výstup na základe vstupu. Spätné šírenie chyby: backpropagation sa vykonáva automaticky pomocou PyTorch.

In [145]:
class MLP(nn.Module):
    def __init__(self, input_size:int, hidden_sizes:list, output_size:int):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        z1 = self.fc1(x)
        a1 = torch.relu(z1)
        z2 = self.fc2(a1)
        a3 = torch.relu(z2)
        z3 = self.fc3(a3)
        y_hat = torch.sigmoid(z3)
        return y_hat



# Nastavenia siete a trénovania

Nastavíme si šírky vrstiev - počet neurónov v každej vrstve.

Pre tréning potrebujeme (a môžeme) definovať:
1. **Kritérium** - Loss funkcia, ktorá nám určí, ako veľmi sme sa priblížili k cieľu.
2. **Optimalizátor** - Algoritmus, ktorý nám pomôže zlepšiť výsledky trénovania. Najzákladnejší je SGD, ale existuje množstvo iných optimalizátorov, ktoré sú vhodné pre rôzne typy sietí. Vyskúšajme si viaceré: SGD, Momentum, Adam, RMSProp, AdamW, ... a ak máte chuť a čas, tak sa môžete pozrieť na AdaHessian 😉
3. **Metriky** - Pomocné metriky, ktoré nám pomôžu sledovať výsledky trénovania.

### Najjednoduchší trénovací cyklus

Bežne vaše trénovacie cykly budú krajšie zabalené, a dáta nebudete môcť načítať celé do modelu. Začnime ale niečím jednoduchším.

In [161]:
layers = [4, 2]
net = MLP(2, layers, 1)
print(net)

criterion = nn.BCELoss()
optimizer_SGD = torch.optim.SGD(net.parameters(), lr=0.03)
precision = metrics.BinaryPrecision()
recall = metrics.BinaryRecall()
f1_score = metrics.BinaryF1Score()

# ------------------------------------------------------------------

x, y = dataset_Circles(1024, radius=0.7, noise=0.2)
inputs = torch.from_numpy(x).float()
labels = torch.from_numpy(y).float()

print(f'{inputs.shape=}')
print(f'{labels.shape=}')

loss_values = []
precision_values = []
recall_values = []
f1_values = []

for epoch in tqdm(range(2000)):  # loop over the dataset multiple times
    # zero the parameter gradients
    optimizer_SGD.zero_grad()

    # forward + backward + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer_SGD.step()

    # print statistics
    loss_values.append(loss.mean().detach().numpy())
    precision_values.append(precision(outputs, labels).detach().numpy())
    recall_values.append(recall(outputs, labels).detach().numpy())
    f1_values.append(f1_score(outputs, labels).detach().numpy())


figure = px.line({'SGD': loss_values, 'Precision': precision_values, 'Recall': recall_values, 'F1': f1_values})
figure.show()

x_val, y_val = dataset_Circles(1024, radius=0.7, noise=0.2)
validation_inputs = torch.from_numpy(x_val).float()

# Predict on validation set

with torch.no_grad():
    outputs = net(validation_inputs)

# Draw validation set
def draw_circles_predicted(x, y):
    xx = x.reshape(-1,2).T
    yy = y.reshape(1,-1)
    fig = px.scatter(x=xx[0], y=xx[1], color=yy[0], width=700, height=700)
    fig.show()

draw_circles_predicted(x_val, outputs.detach().numpy())


MLP(
  (fc1): Linear(in_features=2, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=2, bias=True)
  (fc3): Linear(in_features=2, out_features=1, bias=True)
)
inputs.shape=torch.Size([1024, 2])
labels.shape=torch.Size([1024, 1])


100%|██████████| 2000/2000 [00:11<00:00, 170.00it/s]
